# Initial Setup

In [18]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from IPython.display import Markdown, display, update_display

In [12]:
load_dotenv(override=True)
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_MODEL = "gpt-4o-mini"
openai = OpenAI()

# Data Preparation

In [22]:
class JobSeeker:
    def __init__(self, cv_path: str, job_vacancy_url: str):
        self.cv = self.extract_cv(cv_path)
        self.job_vacancy = self.extract_job_vacancy(job_vacancy_url)

    def extract_cv(self, cv_path: str) -> str:
        print("\nExtracting CV ...")
        try:
            doc = pymupdf.open(cv_path)
            byte_text = b""
            for page in doc:
                byte_text += page.get_text().encode("utf-8")
            print("CV successfully extracted.")
            return byte_text.decode("utf-8")
        except Exception as e:
            print(f"Error reading CV: {e}")
            return ""

    def extract_job_vacancy(self, job_vacancy_url: str) -> str:
        print("\nScraping job vacancy ...")
        chrome_options = uc.ChromeOptions()
        chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 Chrome/123.0.0.0 Safari/537.36")
        chrome_options.headless = True
        try:
            driver = uc.Chrome(options=chrome_options, version_main=135)
            driver.get(job_vacancy_url)
            elements = WebDriverWait(driver, 30).until(
                EC.presence_of_all_elements_located((By.TAG_NAME, "p"))
            )
            text = "".join([el.text for el in elements if el.text!=""])
            print("Job vacancy successfully scraped.")
            driver.quit()
        except Exception as e:
            print(f"Error scraping job vacancy: {e}")
            text = ""
        finally:
            return text

In [23]:
cv_path = "yusup_cv.pdf"
job_vacancy_url = "https://jobs.dell.com/en/job/singapore/machine-learning-engineer/375/75554835840"

yusup = JobSeeker(cv_path, job_vacancy_url)


Extracting CV ...
CV successfully extracted.

Scraping job vacancy ...
Job vacancy successfully scraped.


In [24]:
yusup.cv

" \nYUSUP HIDAYAT WINATA \nML Engineer \n+6281281118430 | yusuphidayatwinata@gmail.com | Indonesia, Jakarta, 14350 \nhttps://www.linkedin.com/in/yusup-winata | Portfolio \n \nPROFESSIONAL SUMMARY \nA passionate Machine Learning Engineer with more than 2 years of experience in AI and ML development, \nmentorship, and program management. Skilled in building and deploying AI models, conducting research, \nand developing user-friendly applications. Proficient in various ML techniques, including classification, \nregression, clustering, NLP, and Computer Vision, with a strong foundation in Deep Learning and Python. \nCommitted to continuous learning and leveraging AI to solve real-world problems. \n \nEXPERIENCES \niOS Developer & ML Engineer Intern, Apple Developer Academy  \n(Mar 2024 - Dec 2024) \n●\u200b Conducted research, including interviews, surveys, and literature reviews, to identify user needs and \npain points that can be addressed through iOS applications. \n●\u200b Designed an

In [25]:
yusup.job_vacancy

'Keyword(s)LocationSINGAPORE, SINGAPOREas rated by current and former employeesMachine Learning Engineer\n\nIn the rapidly moving Artificial Intelligence era, few spaces are moving faster than the AI-enabled PC.  As a leading provider of world-class technology, this means bringing more intelligence into the PC ecosystem, enabling superior performance, enhanced productivity, and delightful experiences while maintaining privacy and security.  We’re developing innovative approaches to introducing intelligence across our client PC portfolio by leveraging current methodologies, models, and tools to develop a robust end-user ecosystem. What’s more, we are collaborating with leading AI technology companies, academics, industry experts, and skilled engineers to deliver cutting-edge solutions that redefine the user experience.Join us to do the best work of your career and make a profound social impact as a Machine Learning Engineer on our Client Solutions Group (CSG) Chief Technology Officer (C

# Prompting

In [52]:
def create_system_prompt() -> str:
    return """You are an expert resume writer who refines, improves, and writes relevant resume based on curriculum vitae and job opening description. \
You are able to write a relevant resume that contains seven sections, including: Header, Professional Summary, Experience, Education, Skills, Tools, and Certifications/Awards. \
The relevant resume you write should fit on 1 page of A4 paper. Respond in markdown."""

In [54]:
def create_user_prompt(cv: str, job_vacancy: str) -> str:
    user_prompt = f"Here is the resume of one of your clients.\n"
    user_prompt += f"{cv}\n\n"
    user_prompt += f"The following is a description of the job opening your client wants to apply for.\n"
    user_prompt += job_vacancy
    return user_prompt

# Call OpenAI API

In [55]:
def write_cv(client: JobSeeker):
    stream = openai.chat.completions.create(
        model=OPENAI_MODEL,
        messages=[
            {"role": "system", "content": create_system_prompt()},
            {"role": "user", "content": create_user_prompt(client.cv, client.job_vacancy)}
          ],
        stream=True
    )

    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ""
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [56]:
write_cv(client=yusup)


# YUSUP HIDAYAT WINATA
**Machine Learning Engineer**  
+6281281118430 | yusuphidayatwinata@gmail.com | Jakarta, Indonesia, 14350  
[LinkedIn](https://www.linkedin.com/in/yusup-winata) | [Portfolio](#)

---

## PROFESSIONAL SUMMARY
Dedicated Machine Learning Engineer with 3+ years of experience in developing and deploying AI-driven solutions, specializing in collaborative project management and mentorship. Proven ability to leverage advanced ML techniques such as classification, regression, NLP, and Computer Vision to deliver innovative applications. Adept at enhancing user experiences through strong UI/UX design principles and data-driven insights. Passionate about making a societal impact by solving real-world problems through technology.

## EXPERIENCE

**iOS Developer & ML Engineer Intern**  
*Apple Developer Academy*  |  *Mar 2024 - Dec 2024*  
- Developed user-friendly iOS applications through deep research into user requirements and pain points.
- Designed and tested dynamic UI/UX using Xcode and MVVM architecture, enhancing user engagement.
- Built and evaluated advanced ML models with Python, TensorFlow, and Keras, improving application functionality based on real user feedback.

**Academic Coordinator & AI Mentor**  
*Orbit Future Academy*  |  *Jul 2021 - Feb 2024*  
- Spearheaded AI and ML programs, collaborating with educational stakeholders to enhance training curricula and content.
- Delivered engaging educational materials in Python, data science, and model deployment, catalyzing learners’ project success.
- Guided students through practical applications of AI, promoting hands-on experience in the field.

## EDUCATION
**Bachelor's Degree in Informatics Engineering**  
*University of Muhammadiyah Jakarta* | *GPA: 3.86/4.00* | *2020*

## SKILLS
- **AI & ML:** Expertise in classification, regression, clustering, NLP, and Computer Vision.  
- **Data Science:** Proficient in data processing, visualization, model training, evaluation, and deployment.  
- **Project Management:** Experience in mentoring, educational content development, and collaborative teamwork.

## TOOLS
Python, TensorFlow, Keras, Scikit-Learn, PyTorch, Streamlit, Flask, Google Colab, Anaconda, Tableau.

## CERTIFICATIONS/AWARDS
- AWS Academy Graduate - AWS Academy Machine Learning Foundations, 2024
- AWS Certified Cloud Practitioner, 2023
- Outstanding Orbitian of The Year (Employee of the Year), 2022 & 2021
- Technology Literacy for Educators - 21st Century Learning Design, 2021

